In [1]:
import pandas as pd
from datetime import datetime

In [20]:
df = pd.read_excel('../raw/Plakate_1945-90_ImportCBS_Teil1.xlsx')
df

,IMD,ERJ,Signatur,Format,Anzahl,Form,Anmerkung,Internvermerk,Trägermaterial,Zustand,kons_bemerkung,Altsignatur,Ländercode,Sprache,Erscheinungsort
0,T/B,1946,1945-90/1,D,1 Blatt,NaN,NaN,NaN,NaN,Risse größer als 3 cm mit drohendem Materialve...,NaN,P65/13,XA-DE,ger,Sowjetische Besatzungszone
1,T/B,1953,1945-90/10,B,1 Blatt,NaN,NaN,NaN,NaN,NaN,NaN,P66/67,XA-DDDE,ger,DDR
2,T,1955,1945-90/100,B,1 Blatt,NaN,NaN,NaN,NaN,stark brüchiges Papier,NaN,P66/96,XA-DDDE,ger,DDR
3,T,1968,1945-90/1000,B,1 Blatt,NaN,Beziehung zu 1945-90/999,NaN,NaN,Verschmutzung,"Verschmutzung auf Vorder- und Rückseite, war v...",P68/76,XA-DDDE,ger,DDR
4,T,1968,1945-90/1001,B,1 Blatt,NaN,Stempel und handschriftliche Anmerkungen auf V...,NaN,NaN,Selbstklebeband,NaN,P68/77,XA-DDDE,ger,DDR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,T,1968,1945-90/995,B,1 Blatt,NaN,NaN,P68/57 und 3x P68/57b als Dubl. entnommen,NaN,NaN,NaN,P68/56,XA-DDDE,ger,DDR
1373,T,1968,1945-90/996,B,1 Blatt,NaN,NaN,2 Dubl. P68/64 entnommen,NaN,NaN,NaN,P68/64,XA-DDDE,ger,DDR
1374,T,1968,1945-90/997,B,1 Blatt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XA-DDDE,ger,DDR
1375,T,1968,1945-90/998,A,1 Blatt,NaN,NaN,NaN,NaN,NaN,NaN,P68/53,XA-DDDE,ger,DDR


In [21]:
def datensatz_schreiben(row):
    pica = """
--0500 Aa Pa, wenn IMD B
--0501 {IMD}
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy\n"""

    if len(str(row.ERJ)) > 0 and len(str(row.ERJ)) < 5:
        # nur ein datum
        pica += f"1100 {row.ERJ}\n1110 *{row.ERJ}$4ezth\n"
    elif len(str(row.ERJ)) == 5 and str(row.ERJ.endswith('?')) == 'True':
        # ein jahr mit fragezeichen
        pica += f"1100 {row.ERJ[:4]}$n[{row.ERJ}]\n1110 {row.ERJ}*{row.ERJ[:4]}$4ezth\n"
    elif len(str(row.ERJ)) > 5 and len(str(row.ERJ)) < 10:
        # zwei daten
        pica += f"1100 {row.ERJ[:4]}$n[zwischen {row.ERJ[:4]} und {row.ERJ[5:]}]\n1110 zwischen {row.ERJ[:4]} und {row.ERJ[5:]}*{row.ERJ}$a{row.ERJ[:4]}$b{row.ERJ[5:]}$4ezth\n"
    else:
        pica += f"\n\n1100 FEHLER {row.ERJ}\n\n"
    
    if pd.isna(row.Trägermaterial):
        pica += "1130 TB-papier\n1130 !040445224!\n"
    else:
        pica += f"1130 {row.Trägermaterial}\n"
    
    if pd.isna(row.Form):
        pica += "1131 !04046198X!\n"
    else:
        pica += f"1131 {row.Form}\n"
    
    pica += "1132 a1-analog;a2-druck;f2-blatt\n"

    if pd.notna(row.Sprache):
        pica += f"1500 /1{row.Sprache}\n"
    
    pica += f"""1700 /1{row.Ländercode}
2105 04,P01
4000 Plakat Nr. {row.Signatur} der Plakatsammlung 1945-1990
4019 Plakat Nr. {row.Signatur} der Plakatsammlung 1945-1990\n"""
    
    if pd.notna(row.Erscheinungsort):
        pica += f"4030 [{row.Erscheinungsort}] : [Verlag nicht ermittelbar]"
    else:
        pica += f"4030 [Erscheinungsort nicht ermittelbar] : [Verlag nicht ermittelbar]"
    
    pica += f"""
4060 {row.Anzahl}
4062 Format {row.Format}
4105 !1238824013!\n"""
    
    if pd.notna(row.Anmerkung):
        pica += f"4201 {row.Anmerkung}\n"
    
    pica += f"4276 nachträglich durch Auswertung der 4201\n"
    
    if pd.notna(row.Internvermerk):
        pica += f"4700 |BSM|{row.Internvermerk}\n"
       
    be_4801 = list()
    for element in [(row.Zustand), (row.kons_bemerkung)]:
        if type(element) == str:
            be_4801.append(element)
        elif type(element) == float:
            pass
        else:
            be_4801.append(f'FEHLER in {row.Signatur}')
    
    if len(be_4801) > 0:
        pica += f"4801 {'; '.join(be_4801)}\n"
    
    pica += f"""5590 [Objektgattung]
5590 !041133579!
6800 [Provenienz]
6800 !004709608! *1961-1993
6800 Stempel
7001 {now.strftime('%d-%m-%y')} : x
7100 GS PL {row.Signatur} @ m
7109 !!DBSM/S!! ; GS PL {row.Signatur}
8034 Dieses Plakat ist Teil eines Erschließungsprojekts und derzeit nicht ausleihbar.\n"""
    if pd.notna(row.Altsignatur):
        pica += f"8598 {row.Altsignatur}\n"
    return pica

In [22]:
now = datetime.now()
with open(f"../dat/plakate1945-90-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/plakate1945-90-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(datensatz_schreiben(row))
            f2.write(datensatz_schreiben(row))

with open(f"../dat/plakate1945-90-recent-sample.dat", 'w') as f:
    for row in df.sample(5).itertuples():
        f.write(datensatz_schreiben(row))

# bekannte Fehler
- ERJ hat manchmal keinen Fehler, was wird standardwert für 1100 und 1110?
- 0500 und 0501 noch nicht geklärt
- 4276